In [23]:
from datetime import datetime

import numpy as np
import pandas as pd

In [24]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADSK', 'AMD', 'AMZN', 'ANET', 'APPH', 
    'AXON', 'BBY','BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD',
    'DDOG', 'DKNG', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'EVGO', 'FTCH',
    'FUV', 'FVRR', 'GH', 'GMED', 'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD',
    'LULU','MDB', 'MEDP', 'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR',
    'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW', 'PINS', 'PTON', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE', 'SHOP', 'SOLO', 'SPLK', 'SQ',
    'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO', 'TWLO', 'TWTR',
    'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [25]:
DL = '/Users/damiansp/Downloads'

In [26]:
!ls $DL

Portfolio_Positions_Feb-18-2022.csv Positions.csv


### E*Trade

In [46]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADSK
AMD


In [28]:
et_df = (
    pd.read_csv(f'{DL}/Positions.csv', index_col=0, skiprows=1)[['Market Value']]
      .rename(columns={'Market Value': 'ET'}))
et_df.index = map(lambda x: x.split()[0], et_df.index)
et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
#et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
ABMD,298.38
ABNB,174.9
AMZN,3052.03
ANET,377.76
BROS,455.93999999999994


In [29]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [30]:
check_symbols(et_df)

All clear


In [31]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,0
ABMD,298
ABNB,175
ADSK,0
AMD,0


### Fidelity

In [32]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Feb-18-2022.csv'

In [33]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,789.39,$1.00,$0.00,$789.39,NaN,NaN,NaN,NaN,1.75%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,3.00,$298.38,-$8.73,$895.14,-$26.19,-2.85%,-$23.64,-2.58%,1.98%,$918.78,$306.26,Cash
2,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,8.00,$174.90,-$8.70,$1399.20,-$69.60,-4.74%,+$30.72,+2.24%,3.10%,$1368.48,$171.06,Cash
3,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,13.00,$113.83,+$1.46,$1479.79,+$18.98,+1.29%,+$159.38,+12.07%,3.28%,$1320.41,$101.57,Cash
4,218520300,ROLLOVER IRA,ANET,ARISTA NETWORKS INC COM USD0.0001,4.00,$125.92,-$3.31,$503.68,-$13.24,-2.57%,+$22.30,+4.63%,1.12%,$481.38,$120.35,Cash


In [50]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [35]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,ABMD,895
2,ROLLOVER IRA,ABNB,1399
3,ROLLOVER IRA,AMD,1480
4,ROLLOVER IRA,ANET,504
5,ROLLOVER IRA,BYND,105


In [36]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
ABMD,895
ABNB,1399
AMD,1480
ANET,504
BYND,105


In [37]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [38]:
for v in accounts.values():
    out = pd.concat([binder, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,0,0,2175,0
ABMD,298,895,298,0
ABNB,175,1399,175,0
ADSK,0,0,0,0
AMD,0,1480,0,0


In [39]:
out.to_csv(f'{DL}/clean.csv')

In [40]:
!open $DL/clean.csv

In [42]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

rm: /Users/damiansp/Downloads/Portfolio_Positions_Feb-18-2022.csv: No such file or directory
rm: /Users/damiansp/Downloads/clean.csv: No such file or directory


### MarketWatch

In [61]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips.csv


In [62]:
mw = pd.read_csv(f'{DL}/Holdings - Damian Satterthwaite-Phillips.csv')[['Symbol', 'Value']]
mw.index = mw.Symbol
mw.drop('Symbol', axis=1, inplace=True)
mw.Value = mw.Value.apply(convert_value)
mw.head()

,Value
Symbol,
AXON,1670
BYND,837
DDOG,2385
DKNG,1141
DOCU,1544


In [63]:
check_symbols(mw)

All clear


In [64]:
out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
out.head()

,Value
AAPL,0
ABMD,0
ABNB,0
ADSK,0
AMD,0


In [65]:
out.to_csv(f'{DL}/mw.csv')

In [66]:
!open $DL/mw.csv

In [68]:
!rm $DL/"Holdings - Damian Satterthwaite-Phillips.csv" $DL/mw.csv
!ls $DL

rm: /Users/damiansp/Downloads/Holdings - Damian Satterthwaite-Phillips.csv: No such file or directory
rm: /Users/damiansp/Downloads/mw.csv: No such file or directory
